In [22]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import os
import matplotlib.pyplot as plt
import io
import csv

from collections import defaultdict
import torch
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist

from langchain_community.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_community.graphs import Neo4jGraph
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI



from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget


from dotenv import load_dotenv
load_dotenv()

/usr/local/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

Prompt for making common_jobs_relationships.csv

Can you make a relationship graph from this list of job titles? Please format the relationship graph so it can be easily read from a file by python. Please only use these values for the relationship graph. Please make names for the connections between job titles like translator related_to tamil teacher and provide the python code for reading the graph into neo4j
Here are the job titles:

In [24]:


def load_data_to_neo4j(uri, user, password, csv_file):
    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        # Create uniqueness constraints for JobTitle nodes with updated syntax
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (n:JobTitle) REQUIRE n.name IS UNIQUE")

        with open(csv_file, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                source = row['Source']
                relationship = row['Relationship']
                target = row['Target']

                # Replace spaces with underscores and convert to uppercase for relationship types
                rel_type = relationship.strip().replace(' ', '_').upper()

                session.run(f"""
                    MERGE (a:JobTitle {{name: $source}})
                    MERGE (b:JobTitle {{name: $target}})
                    MERGE (a)-[r:{rel_type}]->(b)
                """, {'source': source, 'target': target})

    driver.close()
    print("Data loaded successfully into Neo4j.")

if __name__ == "__main__":
    csv_file = 'data/common_jobs_relationships.csv'
    load_data_to_neo4j(os.environ.get("NEO4J_URI"), os.environ.get("NEO4J_USERNAME"), os.environ.get("NEO4J_PASSWORD"), csv_file)

Data loaded successfully into Neo4j.


In [25]:
def show_graph():
    uri = os.environ.get("NEO4J_URI")
    auth = (os.environ.get("NEO4J_USERNAME"), os.environ.get("NEO4J_PASSWORD"))
    driver = GraphDatabase.driver(uri, auth=auth)
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (s) - [r:!MENTIONS] -> (t) RETURN s, r, t").graph())
    widget.node_label_mapping = 'id'
    return widget

show_graph()

GraphWidget(layout=Layout(height='800px', width='100%'))